In [1]:
# Import the necessary information
%store -r louisa_gdf crowley_gdf site_dir data_dir

no stored variable or alias louisa_gdf
no stored variable or alias crowley_gdf
no stored variable or alias site_dir
no stored variable or alias data_dir


In [ ]:
### Import necessary packages
### Reproducable file paths
import os
from glob import glob
import pathlib

### Managing spatial data
import geopandas as gpd
import xrspatial

### Managing other types of data
import numpy as np
import pandas as pd
import rioxarray as rxr
import rioxarray.merge as rmrm

### Manage invalid geometries
from shapely.geometry import MultiPolygon, Polygon

### Visualizing data
import holoviews as hv
import hvplot.pandas
import hvplot.xarray


/Users/erinzimmerman/miniconda3/envs/earth-analytics-python/lib/python3.11/site-packages/dask/dataframe/__init__.py:49: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [ ]:
# Import necessary packages
import os
import pathlib
import pandas as pd # Aggregating, data manipulation
import re # Parsing information out of file names
import rioxarray as rxr # Work with raster data
from rioxarray.merge import merge_arrays # Merge rasters
import geopandas as gpd # Work with vector data
import hvplot.pandas
import xrspatial
from math import floor, ceil
import matplotlib.pyplot as plt
import glob
import warnings

## Soil Data
Soil data was drawn from the POLARIS Database hosted at Duke University. The data repository can be found [here]() 

In [ ]:
# Download soil data
# Define the download URL for the study area (Polaris Data). 
soil1_url_template = ("http://hydrology.cee.duke.edu"
            "/POLARIS/PROPERTIES/v1.0"
            "/ph"
            "/mean"
            "/30_60/"
            "lat{min_lat}{max_lat}_lon{min_lon}{max_lon}.tif"
)

soil1_url = soil1_url_template.format(
        min_lat=46, max_lat=47, min_lon=-96, max_lon=-98)

soil1_url

#Define Bounds for Sheyenne National Grassland
habitat1_bounds = bounds_min_lon, bounds_min_lat, bounds_max_lon, bounds_max_lat = (
    habitat1_gdf.total_bounds)

soil1_url_list = []
for min_lon in range(floor(bounds_min_lon), ceil(bounds_max_lon)):
    for min_lat in range(floor(bounds_min_lat), ceil(bounds_max_lat)):
        soil1_url = soil1_url_template.format(
            min_lat=min_lat, max_lat=min_lat+1,
            min_lon=min_lon, max_lon=min_lon+1)
        soil1_url_list.append(soil1_url)
soil1_url_list

soil1_das = []
#loop through each of the soil files
for i in soil1_url_list:
     # Load the raster data into Python, mask and scale and squeeze
    soil1_da = rxr.open_rasterio(
         i,
         mask_and_scale=True
         ).squeeze()
    print('OPENED ')

    # Crop the raster data
    cropped1_da = soil1_da.rio.clip_box(*habitat1_bounds) 
    soil1_das.append(cropped1_da)
    print('CROPPED')

    # Merge tiles
soil1_merged_das = merge_arrays(soil1_das)

soil1_merged_das.plot()

# Create a plot
fig1, ax = plt.subplots(figsize=(10, 8))

# Plot the raster data (soil1_das)
soil1_merged_das.plot(ax=ax, cmap="viridis", alpha=0.8)  # Adjust alpha for transparency

# Plot the boundaries of the vector data (habitat1_gdf)
habitat1_gdf.boundary.plot(ax=ax, color='purple', linewidth=2)

# Add title and labels
ax.set_title("Overlay of Soil Data and Sheyenne National Grasslands Boundary")
ax.set_xlabel("Longitude")
ax.set_ylabel("Latitude")

# Show the plot
plt.show()

### Topographic data

One way to access reliable elevation data is from the [SRTM
dataset](https://www.earthdata.nasa.gov/data/instruments/srtm),
available through the [earthaccess
API](https://earthaccess.readthedocs.io/en/latest/quick-start/).

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-task"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Try It</div></div><div class="callout-body-container callout-body"><p>Write a <strong>function with a numpy-style docstring</strong> that
will download SRTM elevation data for a particular location and
calculate any additional topographic variables you need such as slope or
aspect.</p>
<p>Then, use loops to download and organize the rasters you will need to
complete this section. Include topographic parameters that will help you
to answer your scientific question.</p></div></div>

> **Warning**
>
> Be careful when computing the slope from elevation that the units of
> elevation match the projection units (e.g. meters and meters, not
> meters and degrees). You will need to project the SRTM data to
> complete this calculation correctly.

In [ ]:
# Download elevation data
# Define data directory for 
habitat1_elevation_dir = os.path.join(project_dir, 'habitat1_elevation_strm')
habitat2_elevation_dir = os.path.join(project_dir, 'habitat2_elevation_strm')


os.makedirs(habitat1_elevation_dir, exist_ok=True)
os.makedirs(habitat2_elevation_dir, exist_ok=True)

In [ ]:
# Log in to earthaccess
earthaccess.login()

In [ ]:
# Search earthaccess for datasets 
datasets = earthaccess.search_datasets(keyword='SRTM DEM')
for dataset in datasets:
    print(dataset['umm']['ShortName'], dataset['umm']['EntryTitle'])

In [ ]:
# Define data search for habitat 1
habitat1_srtm_pattern = os.path.join(habitat1_elevation_dir, '*.hgt.zip') # I feel there is a way to make this a function but I am running out of time

# Define bounds for habitat 2
bounds_habitat1 = tuple(habitat1_gdf.total_bounds)
buffer = .05 
xmin, ymin, xmax, ymax = bounds_habitat1
bounds_habitat1_buffer = (xmin-buffer, ymin-buffer, xmax+buffer, ymax+buffer)

if not glob(habitat1_srtm_pattern):
    habitat1_srtm_results = earthaccess.search_data(
        short_name="SRTMGL1",
        bounding_box=bounds_habitat1_buffer
    )
    habitat1_srtm_results = earthaccess.download(habitat1_srtm_results, habitat1_elevation_dir)
else:
    print("Files already exist. Skipping download.")

In [ ]:
# Create a list
habitat1_srtm_da_list = []
bounds_habitat1_buffer = tuple(habitat1_gdf.total_bounds)
for habitat1_srtm_path in glob(habitat1_srtm_pattern):
    tile_da = rxr.open_rasterio(habitat1_srtm_path, mask_and_scale=True).squeeze()
    
    # Crop data arrays
    habitat1_cropped_da = tile_da.rio.clip_box(*bounds_habitat1_buffer) 
    habitat1_srtm_da_list.append(habitat1_cropped_da)

# Merge tiles
habitat1_srtm_da = rxrmerge.merge_arrays(habitat1_srtm_da_list)

# Initialize a figure and axis
fig, ax = plt.subplots(figsize=(10, 8))  # Adjust figure size as needed

# Plot the DataArray
habitat1_srtm_da.plot(ax=ax)

# Plot the habitat1 boundary on the same axis
habitat1_gdf.boundary.plot(ax=ax, color='yellow', linewidth=0.5)

# Add a title and axis labels
ax.set_title('SRTM Elevation Data with Sheyenne National Grassland', fontsize=14)
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')

# Display the plot
plt.show()

In [ ]:
# Reproject and calculate slope
utm14_epsg = 32614
habitat1_srtm_project_da = habitat1_srtm_da.rio.reproject(32614)
habitat1_proj_gdf = habitat1_gdf.to_crs(utm14_epsg)

# Calculate slope
habitat1_slope_full_da = xrspatial.slope(habitat1_srtm_project_da)

# Clip the slope data by the habitat1 boundaries
habitat1_slope_da = habitat1_slope_full_da.rio.clip(habitat1_proj_gdf.geometry)

# Create a figure and axes to ensure consistent plotting
fig, ax = plt.subplots(figsize=(10, 8))

# Plot the slope data
habitat1_slope_da.plot(ax=ax, cmap='terrain')

# Add a title and labels after plotting
ax.set_title('SRTM Elevation Data with Sheyenne National Grassland', fontsize=16)
ax.set_xlabel('Longitude', fontsize=12)
ax.set_ylabel('Latitude', fontsize=12)

# Show the plot
plt.show()

In [ ]:
# Reproject and calculate slope
utm13_epsg = 32613
habitat2_srtm_project_da = habitat2_srtm_da.rio.reproject(32613)
habitat2_proj_gdf = habitat2_gdf.to_crs(utm13_epsg)

# Calculate slope
habitat2_slope_full_da = xrspatial.slope(habitat2_srtm_project_da)

# Clip the slope data by the habitat2 boundaries
habitat2_slope_da = habitat2_slope_full_da.rio.clip(habitat2_proj_gdf.geometry)

# Create a figure and axes to ensure consistent plotting
fig, ax = plt.subplots(figsize=(10, 8))

# Plot the slope data
habitat2_slope_da.plot(ax=ax, cmap='terrain')

# Add the Habitat 2 boundary on top in yellow (I took out the boundary because it was obscuring a lot of the raster data)
# habitat2_proj_gdf.boundary.plot(ax=ax, color='yellow')

# Add a title and labels after plotting
ax.set_title('SRTM Elevation Data with Little Missouri Grasslands Boundaries', fontsize=16)
ax.set_xlabel('Longitude', fontsize=12)
ax.set_ylabel('Latitude', fontsize=12)

# Show the plot
plt.show()

### Climate model data

You can use MACAv2 data for historical and future climate data. Be sure
to compare at least two 30-year time periods (e.g. historical vs. 10
years in the future) for at least four of the CMIP models. Overall, you
should be downloading at least 8 climate rasters for each of your sites,
for a total of 16. **You will *need* to use loops and/or functions to do
this cleanly!**.

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-task"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Try It</div></div><div class="callout-body-container callout-body"><p>Write a <strong>function with a numpy-style docstring</strong> that
will download MACAv2 data for a particular climate model, emissions
scenario, spatial domain, and time frame. Then, use loops to download
and organize the 16+ rasters you will need to complete this section. The
<a
href="http://thredds.northwestknowledge.net:8080/thredds/reacch_climate_CMIP5_macav2_catalog2.html">MACAv2
dataset is accessible from their Thredds server</a>. Include an
arrangement of sites, models, emissions scenarios, and time periods that
will help you to answer your scientific question.</p></div></div>

In [ ]:
# Download climate data

<link rel="stylesheet" type="text/css" href="./assets/styles.css"><div class="callout callout-style-default callout-titled callout-respond"><div class="callout-header"><div class="callout-icon-container"><i class="callout-icon"></i></div><div class="callout-title-container flex-fill">Reflect and Respond</div></div><div class="callout-body-container callout-body"><p>Make sure to include a description of the climate data and how you
selected your models. Include a citation of the MACAv2 data</p></div></div>

YOUR CLIMATE DATA DESCRIPTION AND CITATIONS HERE

In [ ]:
# Store the essential information to import into the next notebook
%store habitat1_gdf habitat2_gdf soil1_merged_das soil2_merged_das